In [1]:
NUM_NEGATIVE_SAMPLES=20

from random import randint
def N_random_values_in_list(full_list, N):
    x=0
    lower_bound  = 0
    upper_bound = len(full_list)-1
    sample_list=[]
    random_nums=[]
    while x < min(N,len(full_list)):
        random_num = randint(lower_bound, upper_bound) # inclusive range
        if random_num in random_nums:
            continue
        else:
            random_nums.append(random_num)
            x += 1
    return [full_list[i] for i in random_nums]

In [2]:
import torch
import gzip
import numpy as np

def convert_to_list(filename):
    if filename.endswith('gz'):
        with gzip.open(filename,'r')as f:
            text_tokens = f.readlines()
    else:
        with open(filename, 'r') as f:
            text_tokens = f.readlines()
    text_tokens = [token.replace('\n','').split('\t') for token in text_tokens]
    text_tokens = [[token[0], token[1].split(' '), token[2].split(' ')] for token in text_tokens]
                   
    return text_tokens

#Sample:question_id, similar_question_id, negative_question_id
def convert_to_samples(filename):
    my_list=convert_to_list(filename)
    new_samples=[]
    for original_sample in my_list:
        for similar in original_sample[1]:
            random_negative_samples = N_random_values_in_list(original_sample[2],NUM_NEGATIVE_SAMPLES)
            new_samples.append([original_sample[0], similar, random_negative_samples])# change this to include all negative 
                                                                                     # examples later
    return new_samples
def make_lookup_table_for_training_data(filename):
    lookup={}
    text_token_list=convert_to_list(filename)
    for token in text_token_list:
        lookup[token[0]] = {'title':token[1],'question':token[2]}
    return lookup
        
#takes  sample_ids of [[q1,p1,n1],[q2,p2,n2]....]
#outputs titles like [[q1_title, p1_title, n1_title],[q2_title,p2_title,n2_title]...]
def convert_sampleids_to_titles(sample_ids,lookup):
    #each sample_id [question_id, pos_id, [neg_ids]]
    #print type(sample_ids)==list, "first"
   
    titles = []
    for sample_id in sample_ids:
        
         #flatten list: [question_id, pos_id, [neg_ids]] --> [question_id, pos_id, neg_id1, neg_id2, ...]
        sample_id= sample_id[:2]+sample_id[2][:]
        #sample_id : question_id, similar_question_id, negative_question_id
        try:
            titles.append([lookup[str(identity)]['title'] for identity in sample_id])
           # print type(sample_id)==list
        except:
            print sample_id, "is sample id", type(sample_id)==list
    return titles
    

In [3]:
#text_tokenized.txt.gz has id \t title \t question body
text_tokenized='askubuntu/text_tokenized.txt.gz'

#train_random.txt
#(1) the query question ID, (2) the list of similar question IDs, and (3) the list of randomly selected question IDs.
train_random_filename='askubuntu/train_random.txt'

#Each line contains (1) the query question ID, (2) the list of similar question IDs, (3) the list of 20 candidate question IDs and (4) the associated BM25 scores of these questions computed by the Lucene search engine. The second field (the set of similar questions) is a subset of the third field.
dev_filename='askubuntu/dev.txt'
test_filename='askubuntu/test.txt'

train_samples = convert_to_samples(train_random_filename)
dev_samples = convert_to_samples(dev_filename)
test_samples = convert_to_samples(test_filename)

lookup = make_lookup_table_for_training_data(text_tokenized)
train_list = convert_to_list(train_random_filename)
train_titles_only = convert_sampleids_to_titles(train_samples, lookup)

In [4]:
def remove_non_ascii(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

word_embeddings = 'askubuntu/vector/vectors_pruned.200.txt.gz'
f = gzip.open(word_embeddings, 'r')
wv_text = [ ]
lines = f.readlines()
for line in lines:
    wv_text.append(line.strip())

word_to_vec = {}

for line in wv_text:
    parts = line.split()
    word = parts[0]
    vector = np.array([float(v) for v in parts[1:]])
    word_to_vec[word] = vector
f.close()

def extract_features(word):
    try:
        word=remove_non_ascii(word)
        word=word.encode('utf-8')
    except:
        print(word)
    return word_to_vec.get(word,[0.0 for i in range(200)])

In [5]:
def find_maximum_title_and_body_length(lookup_table):
    max_len_title = -1
    max_len_question = -1
    max_len_question_id = 0
    for key, dict_val in lookup_table.iteritems():
        len_title = len(dict_val['title'])
        len_question = len(dict_val['question'])
        if len_title > max_len_title:
             max_len_title = len_title
        if len_question > max_len_question:
            max_len_question = len_question
            max_len_question_id = key
    return max_len_title, max_len_question

def title_to_feature_matrix(title_word_list):
    feature_matrix = []
    for word in title_word_list:
        word_features = extract_features(word)
        feature_matrix.append(word_features)
    #Pad the feature with zeros to ensure all inputs to the net have the same dimension
    feature_matrix += [[0.] * NUM_FEATURES_PER_WORD] * (MAX_TITLE_LENGTH - len(title_word_list))
    #print np.array(feature_matrix).T.shape
    return np.array(feature_matrix).T

In [6]:
MAX_TITLE_LENGTH, MAX_BODY_LENGTH = find_maximum_title_and_body_length(lookup)
NUM_FEATURES_PER_WORD = 200
INPUT_DIM = (MAX_TITLE_LENGTH, NUM_FEATURES_PER_WORD)

In [7]:
import torch.utils.data as data_utils
BATCH_SIZE=50

#for each tuple of titles make a feature vector that is num_titles x 200 x 38
# where num_titles = 1 (target) + 1 (positive) + n (negative) 
features = []
for i in range(len(train_titles_only[:1000])):   # we should include all but this is just for simplicity 
    sample=train_titles_only[i]
    if i%1000==0 or i>9000:
        print i
    
    target_title = sample[0]
    positive_title = sample[1]
    negative_titles = sample[2:]
        
    target_features = title_to_feature_matrix(target_title)
    positive_features = title_to_feature_matrix(positive_title)
    n_negative_features = [title_to_feature_matrix(negative_title) for negative_title in negative_titles]
    
    #all features should be size num_titles x feature size ....
    # where num_titles = 1 (target) + 1 (positive) + n (negative) 
    all_features=[[[0.]*MAX_TITLE_LENGTH] * NUM_FEATURES_PER_WORD]*(2+NUM_NEGATIVE_SAMPLES)

    all_features[0]=target_features
    all_features[1]=positive_features
    all_features[2:len(n_negative_features)+2]=n_negative_features
    features.append(all_features)
print("created features", len(all_features))

targets = torch.LongTensor(torch.FloatTensor(features).size(0), 1).zero_()
train = data_utils.TensorDataset(torch.FloatTensor(features), targets)
print("created features")
train_loader = data_utils.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)

0
('created features', 22)
created features


In [34]:
#array is structured like a batch of features 50x200x38
def find_start_of_padding(batch):
    vec_lengths_in_batch = []
    for batch_num in range(0, len(batch)):
        single_vec = batch[batch_num]
        length = find_start_of_padding_single_vec(single_vec) + 1
        vec_lengths_in_batch.append(length)
    return vec_lengths_in_batch
#batch = 200x38
def find_start_of_padding_single_vec(single_vec):
    for idx in range(len(single_vec[0])-1, -1, -1):
        if single_vec[0][idx] != 0.:
            return idx
    #if the whole sequence is 0s
    return 0

In [27]:
import torch
import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable


class CNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CNN, self).__init__()
        
        self.conv1d = nn.Conv1d(200, 200, KERNEL_SIZE)
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(200, hidden_dim, KERNEL_SIZE),
            nn.ReLU()
        )
    def forward(self, x):
        x = self.layer1(x)
        return x

In [75]:
def feature_vec_to_variable_mean_pool(features):
    vec_lengths_in_batch = find_start_of_padding(features)
    mean_pooled_features = []
    for idx, feature in enumerate(features):
        length = vec_lengths_in_batch[idx]
        #feature is 200x38
        feature = np.array(feature.numpy())/length
        mean_pooled_features.append(feature)
#     print len(mean_pooled_features)
#     print len(mean_pooled_features[0])
#     print len(mean_pooled_features[0][0])
#     print type(mean_pooled_features)
    mean_pooled_features = np.array(mean_pooled_features)
    return Variable(torch.FloatTensor(mean_pooled_features))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
KERNEL_SIZE = 3
INPUT_SIZE = 200
HIDDEN_SIZE = 600
LEARNING_RATE = 1e-3
NUM_EPOCHS = 25
net = CNN(INPUT_SIZE, HIDDEN_SIZE)

criterion = nn.MultiMarginLoss(p=1, margin=0.2, weight=None, size_average=True) #HAHA just put these in to look smart 
optimizer = torch.optim.Adam(net.parameters(), lr = LEARNING_RATE)

print "going in to training"
# ----TRAINING
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
   # for sample in features:
    for i,(sample,label) in enumerate(train_loader):
        if i%25==0:
            print i
        target_features=sample[:,0] 
#         print len(target_features), 'a' #50
#         print len(target_features[0])   #200
#         print len(target_features[0][0])#38
#         print target_features,'d'
        target_features = feature_vec_to_variable_mean_pool(target_features)
                
        positive_features=sample[:,1]
        positive_features=feature_vec_to_variable_mean_pool(positive_features)
        
        n_negative_features=[sample[:,i] for i in range(2, len(sample[0]))]
        n_negative_features=[feature_vec_to_variable_mean_pool(negative_features) for negative_features in n_negative_features]
        
        #print target_features.data.numpy().shape, "is target features shape"
        target_matrix = net(target_features)
        print target_matrix.data.numpy().shape, "is target matrix shape"
        positive_matrix = net(positive_features)
        n_negative_matrices = [net(negative_features) for negative_features in n_negative_features]
        
        target_vec = target_matrix.data.numpy().reshape(50,1,-1)
        #target_vec = [target_matrix.data.numpy()[i].flatten() for i in range(len(target_matrix))]
        print np.array(target_vec).shape, "is target vec shape 50 x 21600"
        #positive_vec = [positive_matrix.data.numpy()[i].flatten() for i in range(len(positive_matrix))]
        positive_vec = positive_matrix.data.numpy().reshape(50,1,-1)
        #n_negative_vecs = [[negative_matrix.data.numpy()[i].flatten() for i in range(len(negative_matrix))] for negative_matrix in n_negative_matrices]
        n_negative_vecs = [negative_matrix.data.numpy().reshape(50,1,-1) for negative_matrix in n_negative_matrices]
        #print target_vec, "is target_vec"
        #print target_vec.shape, "is for target vec"
        #print positive_vec.shape, "is for pos vec"
        cos_sim_positive = [cosine_similarity(target_vec[i], positive_vec[i]) for i in range(BATCH_SIZE)]
        print len(cos_sim_positive), "should be 50"
        print cos_sim_positive[0][0], "number"
        all_cos_sim_negative=[]
        for i in range(BATCH_SIZE):
            batch_of_cos_sims=[]
            for j in range(len(n_negative_vecs)):
                negative_vec=n_negative_vecs[j]
                #print negative_vec[0], "is negative_vec", i
                batch_of_cos_sims.append(cosine_similarity(target_vec[i], negative_vec[i]))
            all_cos_sim_negative.append(batch_of_cos_sims)
        print len(all_cos_sim_negative), all_cos_sim_negative[0]
        all_cos_sim_negative=np.array(all_cos_sim_negative)[:,:,:,0]
        cos_sim_positive = np.array([cos_sim_positive[i] for i in range(BATCH_SIZE)])
        # cos sims should be BATCH_SIZE x num questions (i.e. 2 when only using 1 negative example or 21 when using 20)        
        cos_sims=np.concatenate((cos_sim_positive, all_cos_sim_negative),axis=1)
        print cos_sims[0], "is cos_sims for first in batch"
        max_idx = Variable(torch.FloatTensor(np.array(cos_sims)), requires_grad=True) #use axis = 1 when we use more negative examples later
        max_idx=max_idx[:,:,0]
       # y = Variable(torch.LongTensor([0]),requires_grad=True)
        y = Variable(label)
        y=y[:,0]
        """
        because we know the 0th index in cos_sims is always the example we expect to most closely match 
        the target question
        """

        optimizer.zero_grad()
        
        loss = criterion(max_idx, y)
        #print loss, max_idx, "is loss and max idx"
        loss.backward()
    
        running_loss += loss.data[0]
#         print running_loss
        optimizer.step()

    print "Loss after epoch " + str(epoch) + " :" + str(running_loss)
# ----END TRAINING



going in to training
0
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99401999] number
[[ 0.99401999]
 [ 0.99309349]
 [ 0.99387562]
 [ 0.99209046]
 [ 0.9917829 ]
 [ 0.99317372]
 [ 0.99070299]
 [ 0.99370563]
 [ 0.99385226]
 [ 0.99208176]
 [ 0.99269956]
 [ 0.99293417]
 [ 0.99436748]
 [ 0.99184692]
 [ 0.98691678]
 [ 0.99317467]
 [ 0.99073637]
 [ 0.98781937]
 [ 0.9926734 ]
 [ 0.993312  ]
 [ 0.9928292 ]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.98767686] number
[[ 0.98767686]
 [ 0.98931187]
 [ 0.98883951]
 [ 0.98729157]
 [ 0.98760641]
 [ 0.98714596]
 [ 0.98457766]
 [ 0.98835576]
 [ 0.98572707]
 [ 0.98875993]
 [ 0.98970908]
 [ 0.98935544]
 [ 0.9878509 ]
 [ 0.98975801]
 [ 0.98886836]
 [ 0.98729086]
 [ 0.98683274]
 [ 0.98826599]
 [ 0.98705971]
 [ 0.98751259]
 [ 0.98660183]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1

(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.98679709] number
[[ 0.98679709]
 [ 0.9845776 ]
 [ 0.98775631]
 [ 0.98565435]
 [ 0.98637432]
 [ 0.98747975]
 [ 0.98900962]
 [ 0.98831433]
 [ 0.98511153]
 [ 0.98795325]
 [ 0.9881773 ]
 [ 0.988217  ]
 [ 0.98826027]
 [ 0.98796219]
 [ 0.98527384]
 [ 0.98866063]
 [ 0.98610938]
 [ 0.9867304 ]
 [ 0.9880259 ]
 [ 0.98628718]
 [ 0.98871219]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99208522] number
[[ 0.99208522]
 [ 0.99514228]
 [ 0.99584568]
 [ 0.99445987]
 [ 0.99470729]
 [ 0.99470508]
 [ 0.99350727]
 [ 0.99208337]
 [ 0.99238539]
 [ 0.99158645]
 [ 0.99567437]
 [ 0.99554837]
 [ 0.99535978]
 [ 0.98943925]
 [ 0.99559212]
 [ 0.98991674]
 [ 0.99568534]
 [ 0.99107158]
 [ 0.99149323]
 [ 0.99000651]
 [ 0.9957279 ]] is cos_sims for first in batch
Loss after epoch 0 :3.80133572221
0
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec s

(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99400008] number
[[ 0.99400008]
 [ 0.99304867]
 [ 0.99135691]
 [ 0.99510145]
 [ 0.99597031]
 [ 0.98955739]
 [ 0.99396741]
 [ 0.99408925]
 [ 0.99412698]
 [ 0.99432945]
 [ 0.99496222]
 [ 0.99531323]
 [ 0.98911333]
 [ 0.99241865]
 [ 0.99437803]
 [ 0.9931246 ]
 [ 0.99155164]
 [ 0.99371171]
 [ 0.99272525]
 [ 0.99310887]
 [ 0.99283582]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99023092] number
[[ 0.99023092]
 [ 0.99201989]
 [ 0.9899404 ]
 [ 0.99331331]
 [ 0.99279487]
 [ 0.99090976]
 [ 0.99225938]
 [ 0.99238062]
 [ 0.98776984]
 [ 0.99219143]
 [ 0.9904145 ]
 [ 0.99251354]
 [ 0.99222839]
 [ 0.99022925]
 [ 0.99184048]
 [ 0.99164999]
 [ 0.99266088]
 [ 0.99131596]
 [ 0.99152452]
 [ 0.99026394]
 [ 0.99218416]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.

(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99478984] number
[[ 0.99478984]
 [ 0.99270475]
 [ 0.99156207]
 [ 0.99206269]
 [ 0.99139827]
 [ 0.99238026]
 [ 0.992791  ]
 [ 0.99293441]
 [ 0.99182326]
 [ 0.99104208]
 [ 0.99123573]
 [ 0.99221766]
 [ 0.99248576]
 [ 0.99241376]
 [ 0.99342024]
 [ 0.99165535]
 [ 0.99305165]
 [ 0.99256074]
 [ 0.99219143]
 [ 0.99257958]
 [ 0.99252099]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99344373] number
[[ 0.99344373]
 [ 0.99083489]
 [ 0.99332309]
 [ 0.99288547]
 [ 0.99384892]
 [ 0.99018562]
 [ 0.99306929]
 [ 0.99440837]
 [ 0.99388766]
 [ 0.99023569]
 [ 0.99240184]
 [ 0.99338871]
 [ 0.99220335]
 [ 0.99182045]
 [ 0.993451  ]
 [ 0.99276471]
 [ 0.99288207]
 [ 0.9934572 ]
 [ 0.99017382]
 [ 0.99386525]
 [ 0.99415028]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.

(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99280894] number
[[ 0.99280894]
 [ 0.99179083]
 [ 0.9919821 ]
 [ 0.99236697]
 [ 0.99140131]
 [ 0.99213302]
 [ 0.99331057]
 [ 0.9892112 ]
 [ 0.99240404]
 [ 0.99399054]
 [ 0.99367952]
 [ 0.99323392]
 [ 0.98875678]
 [ 0.99320632]
 [ 0.99440289]
 [ 0.99281549]
 [ 0.99210936]
 [ 0.99126595]
 [ 0.99442536]
 [ 0.99441803]
 [ 0.99311489]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99646538] number
[[ 0.99646538]
 [ 0.99347311]
 [ 0.99253482]
 [ 0.99530458]
 [ 0.99525636]
 [ 0.99489355]
 [ 0.99210054]
 [ 0.99479389]
 [ 0.99471647]
 [ 0.99479973]
 [ 0.99556255]
 [ 0.9924261 ]
 [ 0.99037784]
 [ 0.99077588]
 [ 0.99350607]
 [ 0.99148637]
 [ 0.99617797]
 [ 0.99543566]
 [ 0.99394274]
 [ 0.99044836]
 [ 0.99267077]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.

(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99238586] number
[[ 0.99238586]
 [ 0.99157083]
 [ 0.99285656]
 [ 0.99193382]
 [ 0.99235415]
 [ 0.99215317]
 [ 0.9934628 ]
 [ 0.99195576]
 [ 0.99314225]
 [ 0.99267566]
 [ 0.99343777]
 [ 0.99022335]
 [ 0.98856562]
 [ 0.99106002]
 [ 0.99322754]
 [ 0.99047709]
 [ 0.9904263 ]
 [ 0.99224782]
 [ 0.99341488]
 [ 0.99175096]
 [ 0.9940691 ]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99182969] number
[[ 0.99182969]
 [ 0.98755878]
 [ 0.99366283]
 [ 0.99162579]
 [ 0.99263662]
 [ 0.99120188]
 [ 0.99168646]
 [ 0.99236923]
 [ 0.990058  ]
 [ 0.99261677]
 [ 0.99259591]
 [ 0.99318123]
 [ 0.99269408]
 [ 0.99288911]
 [ 0.99253863]
 [ 0.99195099]
 [ 0.99207532]
 [ 0.99235308]
 [ 0.99281925]
 [ 0.99184608]
 [ 0.99247313]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.

(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99606073] number
[[ 0.99606073]
 [ 0.99089503]
 [ 0.9887231 ]
 [ 0.99162698]
 [ 0.99406558]
 [ 0.99488348]
 [ 0.99564791]
 [ 0.99250865]
 [ 0.98977095]
 [ 0.99318314]
 [ 0.99044204]
 [ 0.99399209]
 [ 0.98851085]
 [ 0.98979533]
 [ 0.99174035]
 [ 0.9942981 ]
 [ 0.99135488]
 [ 0.99466592]
 [ 0.9899984 ]
 [ 0.99568838]
 [ 0.98954272]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.9944824] number
[[ 0.9944824 ]
 [ 0.9938007 ]
 [ 0.9929204 ]
 [ 0.99495447]
 [ 0.99228561]
 [ 0.99462074]
 [ 0.99508983]
 [ 0.99145997]
 [ 0.99365515]
 [ 0.99329954]
 [ 0.99572623]
 [ 0.99347591]
 [ 0.99311423]
 [ 0.99604678]
 [ 0.99394304]
 [ 0.9926033 ]
 [ 0.99397898]
 [ 0.99497914]
 [ 0.99271405]
 [ 0.9948107 ]
 [ 0.99214935]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.9

(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99029875] number
[[ 0.99029875]
 [ 0.99461263]
 [ 0.99621582]
 [ 0.99089289]
 [ 0.99199283]
 [ 0.99287713]
 [ 0.99413496]
 [ 0.99571908]
 [ 0.99595076]
 [ 0.99574369]
 [ 0.99527276]
 [ 0.99664611]
 [ 0.99176311]
 [ 0.99458468]
 [ 0.99224067]
 [ 0.99616909]
 [ 0.99205202]
 [ 0.99545276]
 [ 0.99102211]
 [ 0.99373758]
 [ 0.99545884]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99528778] number
[[ 0.99528778]
 [ 0.99466592]
 [ 0.99305379]
 [ 0.98851085]
 [ 0.9938857 ]
 [ 0.99262965]
 [ 0.99032545]
 [ 0.98914158]
 [ 0.99488348]
 [ 0.99523497]
 [ 0.99414724]
 [ 0.99445283]
 [ 0.99284023]
 [ 0.99447685]
 [ 0.99402106]
 [ 0.99174035]
 [ 0.99347377]
 [ 0.99238408]
 [ 0.99313104]
 [ 0.99509513]
 [ 0.99270952]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.

(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.9924525] number
[[ 0.9924525 ]
 [ 0.99445915]
 [ 0.99426329]
 [ 0.99473113]
 [ 0.99359637]
 [ 0.99395287]
 [ 0.99391472]
 [ 0.99395829]
 [ 0.9935931 ]
 [ 0.992401  ]
 [ 0.9940359 ]
 [ 0.99404931]
 [ 0.99413341]
 [ 0.99335498]
 [ 0.9950819 ]
 [ 0.99343318]
 [ 0.99080014]
 [ 0.99357957]
 [ 0.99211836]
 [ 0.9933486 ]
 [ 0.99418598]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99098706] number
[[ 0.99098706]
 [ 0.9920488 ]
 [ 0.99303877]
 [ 0.99336416]
 [ 0.99193281]
 [ 0.99032855]
 [ 0.99375153]
 [ 0.99195087]
 [ 0.99231148]
 [ 0.98732305]
 [ 0.99270105]
 [ 0.99464738]
 [ 0.99160421]
 [ 0.99115145]
 [ 0.98916459]
 [ 0.99352098]
 [ 0.9911465 ]
 [ 0.99397081]
 [ 0.99318683]
 [ 0.99377155]
 [ 0.9947927 ]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.9

(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99643242] number
[[ 0.99643242]
 [ 0.99394417]
 [ 0.9943828 ]
 [ 0.99552667]
 [ 0.99332893]
 [ 0.99657738]
 [ 0.99683106]
 [ 0.99493313]
 [ 0.99422514]
 [ 0.99510056]
 [ 0.99515665]
 [ 0.99510616]
 [ 0.99462283]
 [ 0.99479818]
 [ 0.99374288]
 [ 0.99409688]
 [ 0.99492544]
 [ 0.99384946]
 [ 0.99455917]
 [ 0.9955973 ]
 [ 0.9953028 ]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.99542928] number
[[ 0.99542928]
 [ 0.99068695]
 [ 0.99378276]
 [ 0.99678248]
 [ 0.99306387]
 [ 0.99481714]
 [ 0.99319482]
 [ 0.99415231]
 [ 0.99297929]
 [ 0.99553466]
 [ 0.995233  ]
 [ 0.99615216]
 [ 0.99419528]
 [ 0.99547946]
 [ 0.9943049 ]
 [ 0.99520171]
 [ 0.99504107]
 [ 0.9930222 ]
 [ 0.99551618]
 [ 0.99384999]
 [ 0.99589038]] is cos_sims for first in batch
(50, 600, 36) is target matrix shape
(50, 1, 21600) is target vec shape 50 x 21600
50 should be 50
[ 0.

In [ ]:
len(train_titles_only)

In [ ]:
criterion(Variable(torch.FloatTensor([1,2,3,4,5,6,7,8,9,10,10.2])),Variable(torch.LongTensor([10])))

In [ ]:
sample[0][0:3]

In [ ]:
cosine_similarity(sample[0][0].numpy().flatten(), sample[0][1].numpy().flatten())[0][0]

In [ ]:
sample[0][0].numpy().shape